# SQL


In [ ]:
import qrcode
import os

def generate_qr_codes(base_url, route_id, stop_count, out_dir="qrcodes"):
    os.makedirs(out_dir, exist_ok=True)

    for stop_order in range(1, stop_count + 1):
        url = f"{base_url}/routes/{route_id}/stop/{stop_order}"
        img = qrcode.make(url)
        filename = os.path.join(out_dir, f"route{route_id}_stop{stop_order}.png")
        img.save(filename)
        print("已產生:", filename, "→", url)

# 使用範例
if __name__ == "__main__":
    # 你的 ngrok host，或正式網域
    base = "https://f1c2fbf338af.ngrok-free.app"
    generate_qr_codes(base_url=base, route_id=2, stop_count=8)


In [1]:
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from urllib.parse import quote
from dotenv import load_dotenv
import requests, os, json, hashlib

load_dotenv()

# ==== 環境參數 ====
MERCHANT_ID = os.getenv("MERCHANT_ID", "123")   # RG商代
TERMINAL_ID = os.getenv("TERMINAL_ID", "123")   # RG端代
STORE_CODE  = os.getenv("STORE_CODE", "123")    # 店家識別碼
KEY_HEX     = os.getenv("KEY", "")              # 端末設備Key (hex)
IV_HEX      = os.getenv("IV", "")               # 端末設備IV (hex)
LAYMON      = os.getenv("LAYMON", "api.test.com")  # 雷門支付 API 主機

KEY = bytes.fromhex(KEY_HEX)
IV  = bytes.fromhex(IV_HEX)

# ==== 測試交易資料 ====
payload = {
    "set_price": "100",
    "pos_order_number": "TEST12345",   # 自訂訂單編號，不可重複
    "callback_url": "https://yourdomain.com/callback",
    "return_url": "https://yourdomain.com/return"
}

# --- AES256-CBC 加解密 ---
def pad(s):
    pad_len = 16 - (len(s) % 16)
    return s + chr(pad_len) * pad_len

def unpad(s):
    return s[:-ord(s[-1])]

def encrypt_aes(data: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(data).encode("utf-8"))
    return b64encode(ct_bytes).decode("utf-8")

def decrypt_aes(enc: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = cipher.decrypt(b64decode(enc))
    return unpad(pt.decode("utf-8"))

# --- 加密交易資料 ---
json_str = json.dumps(payload, separators=(',', ':'))
TransactionData = encrypt_aes(json_str, KEY, IV)
HashDigest = hashlib.sha256(TransactionData.encode("utf-8")).hexdigest()

print("加密後 TransactionData:", TransactionData)
print("校驗碼 HashDigest:", HashDigest)

# --- 發送 API (線上付款) ---
url = f"https://{LAYMON}/calc/pay_encrypt/{STORE_CODE}"
params = {
    "TransactionData": quote(TransactionData),  # URL encode
    "HashDigest": HashDigest
}

print("請求網址：", url)
print("GET 參數：", params)

# ⚠️ 測試時建議先只列印，不要真的發送
# response = requests.get(url, params=params)
# print(response.text)


加密後 TransactionData: LwIo/f0GsJQ1mLLlWKTqJsoi5qsxSApZKfjpXToaReHNFDMcw560TLVvIOVcIt6NDQyc8YwnCu8tECEJaTJqG1FP1rI/0wYlJeQR1p1Dw8CqA84ySf6IrrHROz3beuJ3QiZF7sYbCxglkz/jbpr9RxfUrt9RQQ8zf3ZmAcWLeMs9J6ryCBIfx+p3XWEk2T0jXyFz/um3DAOK9miKcbQpeA==
校驗碼 HashDigest: 98c15532d725f6135887ce75b867333697f89406bd253fe25192768854434556
請求網址： https://iqrc.epay365.com.tw/calc/pay_encrypt/e4f76eb8-d236-45fb-984b-92b2d1912492
GET 參數： {'TransactionData': 'LwIo/f0GsJQ1mLLlWKTqJsoi5qsxSApZKfjpXToaReHNFDMcw560TLVvIOVcIt6NDQyc8YwnCu8tECEJaTJqG1FP1rI/0wYlJeQR1p1Dw8CqA84ySf6IrrHROz3beuJ3QiZF7sYbCxglkz/jbpr9RxfUrt9RQQ8zf3ZmAcWLeMs9J6ryCBIfx%2Bp3XWEk2T0jXyFz/um3DAOK9miKcbQpeA%3D%3D', 'HashDigest': '98c15532d725f6135887ce75b867333697f89406bd253fe25192768854434556'}
